In [1]:
import pandas as pd
df = pd.read_parquet('data/short_summary/combined.parquet', engine='pyarrow')

In [2]:
from src.feature_eng import extract_and_format_date
df = extract_and_format_date(df)

In [3]:
df

,identifier,event_id,event_name,venue,calculate_date,price_level_name,cap,revenue_to_date,s/t-rate,opens,...,pio_opportunity,host_event_code,archtics_event_name,dl_rowhash,dl_iscurrent,dl_isdeleted,dl_recordstartdateutc,dl_recordenddateutc,dl_load_id,event_date
0,AllData_11005B3CECED2D64_Rogers Arena_Canucks_...,11005B3CECED2D64,Canucks vs. Vegas Golden Knights,Rogers Arena,11/08/2021,4: 4/3,64,0.0,0.0,0,...,0.0,ECNK0403,None,e8fb4d2f929a11b206bf3e613a3f93382f1a68eaa79731...,True,False,2024-04-11T09:59:14.544526-07:00,NaN,0,2022-04-03
1,AllData_11005B3CECED2D64_Rogers Arena_Canucks_...,11005B3CECED2D64,Canucks vs. Vegas Golden Knights,Rogers Arena,11/07/2021,4: 4/3,64,0.0,0.0,0,...,0.0,ECNK0403,None,0568e792ef23479eaeab97b8c61d0f254fa230f9af9b62...,True,False,2024-04-11T09:59:14.544526-07:00,NaN,0,2022-04-03
2,AllData_11005B3CECED2D64_Rogers Arena_Canucks_...,11005B3CECED2D64,Canucks vs. Vegas Golden Knights,Rogers Arena,11/11/2021,4: 4/3,64,0.0,0.0,0,...,0.0,ECNK0403,None,a3f1d09c760f67908bf4ee53f09b87ad9999787f17189f...,True,False,2024-04-11T09:59:14.544526-07:00,NaN,0,2022-04-03
3,AllData_11005B3CECED2D64_Rogers Arena_Canucks_...,11005B3CECED2D64,Canucks vs. Vegas Golden Knights,Rogers Arena,11/09/2021,4: 4/3,64,0.0,0.0,0,...,0.0,ECNK0403,None,67c7ef71ba3395b6780ca30b1d039e74a4f63b3bb029f2...,True,False,2024-04-11T09:59:14.544526-07:00,NaN,0,2022-04-03
4,AllData_11005B3CECED2D64_Rogers Arena_Canucks_...,11005B3CECED2D64,Canucks vs. Vegas Golden Knights,Rogers Arena,11/10/2021,4: 4/3,64,0.0,0.0,0,...,0.0,ECNK0403,None,098a7d70b99caf92c4c90a6321f339282ffdc8b9394838...,True,False,2024-04-11T09:59:14.544526-07:00,NaN,0,2022-04-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748891,AllData_11005D05E8F553A7_Rogers Arena_Canucks_...,11005D05E8F553A7,Canucks vs. Seattle Kraken,Rogers Arena,08/16/2022,28: 28/R,757,0.0,0.0,0,...,0.0,ECAN1222,None,d0b754844b56cf34220c04e3901aef2d40a0870d166e6e...,True,False,2024-04-11T09:59:14.544526-07:00,NaN,0,2022-12-22
748892,AllData_11005D05E8F553A7_Rogers Arena_Canucks_...,11005D05E8F553A7,Canucks vs. Seattle Kraken,Rogers Arena,08/16/2022,29: 29/S,176,0.0,0.0,0,...,0.0,ECAN1222,None,1afa65a68b4e72e3c4a1ea4f6c53c6dc7af47beb4f77f8...,True,False,2024-04-11T09:59:14.544526-07:00,NaN,0,2022-12-22
748893,AllData_11005D05E8F553A7_Rogers Arena_Canucks_...,11005D05E8F553A7,Canucks vs. Seattle Kraken,Rogers Arena,08/16/2022,30: 30/T,76,0.0,0.0,0,...,0.0,ECAN1222,None,8a4088266ede8afe31f1a585a40ed3e17c7692cdc54b59...,True,False,2024-04-11T09:59:14.544526-07:00,NaN,0,2022-12-22
748894,AllData_11005D05E8F553A7_Rogers Arena_Canucks_...,11005D05E8F553A7,Canucks vs. Seattle Kraken,Rogers Arena,08/16/2022,31: 31/U,28,0.0,0.0,0,...,0.0,ECAN1222,None,920ea28be7a904070c6233ac3a72e63a51cc87869f8f74...,True,False,2024-04-11T09:59:14.544526-07:00,NaN,0,2022-12-22


In [4]:
grouped_df = df.groupby(['price_level_name', 'event_date'])['initial_price'].mean().reset_index()
filtered_df = grouped_df[grouped_df['event_date'] >= '2021-07-01']
filtered_df

,price_level_name,event_date,initial_price
0,10: 10/9,2021-10-03,64.500000
1,10: 10/9,2021-10-05,77.357143
2,10: 10/9,2021-10-09,83.590909
3,10: 10/9,2021-10-26,147.714286
4,10: 10/9,2021-10-28,94.500000
...,...,...,...
5979,9: 9/A,2022-03-12,92.977778
5980,9: 9/A,2022-03-13,84.246914
5981,9: 9/A,2022-03-14,150.060127
5982,9: 9/A,2022-03-15,114.121212


In [6]:
import altair as alt

# Create a selection that allows multiple selections in the legend
selection = alt.selection_point(fields=['price_level_name'], bind='legend')

# Define the line chart
chart = alt.Chart(filtered_df).mark_line(point=True).encode(
    x=alt.X('event_date:T', title='Event Date'),
    y=alt.Y('initial_price:Q', title='Initial Price'),
    color=alt.Color('price_level_name:N', title='Price Level Name', legend=alt.Legend(title="Select Price Level")),
    tooltip=[alt.Tooltip('event_date:T', title='Event Date'), 
             alt.Tooltip('initial_price:Q', title='Initial Price'),
             alt.Tooltip('price_level_name:N', title='Price Level Name')],
    opacity=alt.condition(selection, alt.value(1), alt.value(0))
).add_params(
    selection
).properties(
    title='Interactive Price Trends by Price Level Over Time',
    width=800,
    height=400
)

chart.display()

alt.Chart(...)